In [1]:
import pandas as pd
import numpy as np
import warnings
from tqdm import tqdm
import datetime
import time
import os
warnings.filterwarnings("ignore")

In [2]:
def mkdir(path):
 
    # 去除首位空格
    path=path.strip()
    # 去除尾部 \ 符号
    path=path.rstrip("\\")
 
    # 判断路径是否存在
    # 存在     True
    # 不存在   False
    isExists=os.path.exists(path)
    
    # 判断结果
    if not isExists:
        # 如果不存在则创建目录
        # 创建目录操作函数
        os.makedirs(path) 
 
        print(path+' 创建成功')
        return True
    else:
        # 如果目录存在则不创建，并提示目录已存在
        print(path+' 目录已存在')
        return False

In [3]:
i = datetime.datetime.now()
time_str = str(i.year) + '_' + str(i.month) + '_' + str(i.day) + '_hour' + str(i.hour)+ '(half)'
# 定义要创建的文件夹
mkpath = '.\\' + time_str 
# 调用函数
mkdir(mkpath)

.\2022_6_30_hour10(half) 创建成功


True

# 读取

In [4]:
path_dc = './DataCleaning_quarterTwo_1.csv'
dc = pd.read_csv(path_dc, index_col=[0])
print(dc.info(null_counts = True))
dc[:2]

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1827023 entries, 2420200000050231 to 442010000019652554
Data columns (total 29 columns):
 #   Column      Non-Null Count    Dtype  
---  ------      --------------    -----  
 0   jjhklb      1827023 non-null  int64  
 1   hydm        1827023 non-null  object 
 2   zczb        1827023 non-null  float64
 3   jyzt        1827023 non-null  float64
 4   qylx        1827023 non-null  int64  
 5   tslx        1827023 non-null  int64  
 6   new         1827023 non-null  object 
 7   time_scale  1827023 non-null  int64  
 8   city        1827023 non-null  object 
 9   three       1827023 non-null  int64  
 10  scale       1827023 non-null  int64  
 11  foreign     1827023 non-null  int64  
 12  X1_2        1827023 non-null  int64  
 13  X3          1827023 non-null  int64  
 14  X4          1827023 non-null  int64  
 15  X5          1827023 non-null  float64
 16  X6          1827023 non-null  float64
 17  X7          1827023 non-null  float64
 

,jjhklb,hydm,zczb,jyzt,qylx,tslx,new,time_scale,city,three,...,X9,X10,X11,X12,X13,X14,X15,X16,X17,X18
nbxh,,,,,,,,,,,,,,,,,,,,,
2420200000050231,3,R,50.0,1.0,4540,6,2007-04,4,黄石市,3,...,0,0.0,0.0,0.0,0.0,0.0,0,0,0,0
2422800000117935,3,Z,1.0,1.0,2130,6,2014-01,3,恩施州,3,...,0,0.0,0.0,0.0,0.0,0.0,0,0,0,0


> 表中`jyzt`为1的是存量企业

> 需要在代码一中加入    
    - `tslx(C|2)`来标明注吊销迁入迁出数据
    >> 注销（07）、吊销（11）、迁入（09）、迁出（13）    
    - `new`来标明新设企业    
    >> *half = ['2022-01', '2022-02', '2022-03', '2022-04', '2022-05', '2022-06']* 这些设置为`1`，其他的设置为`0`

In [5]:
new_month = ['2022-04', '2022-05', '2022-06']
for i in new_month:
    dc.loc[dc['new'] == i, 'new'] = 1

dc.loc[dc['new'] != 1, 'new'] = 0
print(dc['new'].value_counts())

0    1724343
1     102680
Name: new, dtype: int64


# unit_df

In [6]:
def series2df(series_name, col_name1, col_name2):
    df_temp = {col_name1:series_name.index, col_name2:series_name.values}
    df_temp = pd.DataFrame(df_temp)
    df_temp.set_index(col_name1, inplace=True)
    return df_temp

In [7]:
def df_combine(df_name, series_name, col_name1, col_name2):
    df_temp = series2df(series_name, col_name1, col_name2)
    combine_result = pd.concat([df_name, df_temp], axis=1)
    return combine_result

# 不同行业下

## 新设企业

In [8]:
industry_new = dc[dc['new'] == 1].groupby(['hydm']).size()
industry_temp = series2df(industry_new, 'hydm', '新设企业')
industry_all = industry_temp
industry_all

,新设企业
hydm,
A,6998
B,62
C,6725
D,414
E,12291
F,35081
G,2077
H,2034
I,7537


## 存量企业
存量企业包括了在这半年中注吊销和迁出的企业

In [9]:
industry_live = dc.groupby(['hydm']).size()
industry_all = df_combine(industry_all, industry_live, 'hydm', '存量企业(包含期间内的注吊销&迁出)')
industry_all

,新设企业,存量企业(包含期间内的注吊销&迁出)
hydm,,
A,6998.0,83587
B,62.0,4053
C,6725.0,142456
D,414.0,10656
E,12291.0,249286
F,35081.0,551560
G,2077.0,51460
H,2034.0,32671
I,7537.0,90482


In [10]:
industry_live = dc[dc['jyzt'] == 1].groupby(['hydm']).size()
industry_all = df_combine(industry_all, industry_live, 'hydm', '存量企业(不包含期间内的注吊销&迁出)')
industry_all

,新设企业,存量企业(包含期间内的注吊销&迁出),存量企业(不包含期间内的注吊销&迁出)
hydm,,,
A,6998.0,83587,82125
B,62.0,4053,3995
C,6725.0,142456,140150
D,414.0,10656,10477
E,12291.0,249286,245601
F,35081.0,551560,532829
G,2077.0,51460,50418
H,2034.0,32671,31774
I,7537.0,90482,88251


## 注吊销企业

In [11]:
industry_zdx = dc[(dc['tslx'] == 7) | (dc['tslx'] == 11)].groupby(['hydm']).size()
industry_all = df_combine(industry_all, industry_zdx, 'hydm', '注吊销企业')
industry_all

,新设企业,存量企业(包含期间内的注吊销&迁出),存量企业(不包含期间内的注吊销&迁出),注吊销企业
hydm,,,,
A,6998.0,83587,82125,1461.0
B,62.0,4053,3995,58.0
C,6725.0,142456,140150,2301.0
D,414.0,10656,10477,178.0
E,12291.0,249286,245601,3663.0
F,35081.0,551560,532829,18703.0
G,2077.0,51460,50418,1041.0
H,2034.0,32671,31774,896.0
I,7537.0,90482,88251,2215.0


### 注销

In [12]:
industry_zx = dc[dc['tslx'] == 7].groupby(['hydm']).size()
industry_all = df_combine(industry_all, industry_zx, 'hydm', '注销企业')
industry_all

,新设企业,存量企业(包含期间内的注吊销&迁出),存量企业(不包含期间内的注吊销&迁出),注吊销企业,注销企业
hydm,,,,,
A,6998.0,83587,82125,1461.0,1396.0
B,62.0,4053,3995,58.0,54.0
C,6725.0,142456,140150,2301.0,2086.0
D,414.0,10656,10477,178.0,167.0
E,12291.0,249286,245601,3663.0,3334.0
F,35081.0,551560,532829,18703.0,16448.0
G,2077.0,51460,50418,1041.0,959.0
H,2034.0,32671,31774,896.0,805.0
I,7537.0,90482,88251,2215.0,1908.0


### 吊销

In [13]:
industry_dx = dc[dc['tslx'] == 11].groupby(['hydm']).size()
industry_all = df_combine(industry_all, industry_dx, 'hydm', '吊销企业')
industry_all

,新设企业,存量企业(包含期间内的注吊销&迁出),存量企业(不包含期间内的注吊销&迁出),注吊销企业,注销企业,吊销企业
hydm,,,,,,
A,6998.0,83587,82125,1461.0,1396.0,65.0
B,62.0,4053,3995,58.0,54.0,4.0
C,6725.0,142456,140150,2301.0,2086.0,215.0
D,414.0,10656,10477,178.0,167.0,11.0
E,12291.0,249286,245601,3663.0,3334.0,329.0
F,35081.0,551560,532829,18703.0,16448.0,2255.0
G,2077.0,51460,50418,1041.0,959.0,82.0
H,2034.0,32671,31774,896.0,805.0,91.0
I,7537.0,90482,88251,2215.0,1908.0,307.0


## 迁入迁出企业

In [14]:
industry_qrc = dc[(dc['tslx'] == 9) | (dc['tslx'] == 13)].groupby(['hydm']).size()
industry_all = df_combine(industry_all, industry_qrc, 'hydm', '迁入迁出企业')
industry_all

,新设企业,存量企业(包含期间内的注吊销&迁出),存量企业(不包含期间内的注吊销&迁出),注吊销企业,注销企业,吊销企业,迁入迁出企业
hydm,,,,,,,
A,6998.0,83587,82125,1461.0,1396.0,65.0,198
B,62.0,4053,3995,58.0,54.0,4.0,14
C,6725.0,142456,140150,2301.0,2086.0,215.0,1097
D,414.0,10656,10477,178.0,167.0,11.0,30
E,12291.0,249286,245601,3663.0,3334.0,329.0,800
F,35081.0,551560,532829,18703.0,16448.0,2255.0,3411
G,2077.0,51460,50418,1041.0,959.0,82.0,204
H,2034.0,32671,31774,896.0,805.0,91.0,103
I,7537.0,90482,88251,2215.0,1908.0,307.0,499


### 迁入

In [15]:
industry_qr = dc[dc['tslx'] == 9].groupby(['hydm']).size()
industry_all = df_combine(industry_all, industry_qr, 'hydm', '迁入企业')
industry_all

,新设企业,存量企业(包含期间内的注吊销&迁出),存量企业(不包含期间内的注吊销&迁出),注吊销企业,注销企业,吊销企业,迁入迁出企业,迁入企业
hydm,,,,,,,,
A,6998.0,83587,82125,1461.0,1396.0,65.0,198,197
B,62.0,4053,3995,58.0,54.0,4.0,14,14
C,6725.0,142456,140150,2301.0,2086.0,215.0,1097,1092
D,414.0,10656,10477,178.0,167.0,11.0,30,29
E,12291.0,249286,245601,3663.0,3334.0,329.0,800,778
F,35081.0,551560,532829,18703.0,16448.0,2255.0,3411,3383
G,2077.0,51460,50418,1041.0,959.0,82.0,204,203
H,2034.0,32671,31774,896.0,805.0,91.0,103,102
I,7537.0,90482,88251,2215.0,1908.0,307.0,499,483


### 迁出

In [16]:
industry_qc = dc[dc['tslx'] == 13].groupby(['hydm']).size()
industry_all = df_combine(industry_all, industry_qc, 'hydm', '迁出企业')
industry_all

,新设企业,存量企业(包含期间内的注吊销&迁出),存量企业(不包含期间内的注吊销&迁出),注吊销企业,注销企业,吊销企业,迁入迁出企业,迁入企业,迁出企业
hydm,,,,,,,,,
A,6998.0,83587,82125,1461.0,1396.0,65.0,198,197,1.0
B,62.0,4053,3995,58.0,54.0,4.0,14,14,NaN
C,6725.0,142456,140150,2301.0,2086.0,215.0,1097,1092,5.0
D,414.0,10656,10477,178.0,167.0,11.0,30,29,1.0
E,12291.0,249286,245601,3663.0,3334.0,329.0,800,778,22.0
F,35081.0,551560,532829,18703.0,16448.0,2255.0,3411,3383,28.0
G,2077.0,51460,50418,1041.0,959.0,82.0,204,203,1.0
H,2034.0,32671,31774,896.0,805.0,91.0,103,102,1.0
I,7537.0,90482,88251,2215.0,1908.0,307.0,499,483,16.0


## 保存

In [17]:
industry_all.fillna(value = 0).to_excel('./' + time_str + '/不同行业下细分.xlsx')

# 不同地区下细分

## 新设企业

In [18]:
city_new = dc[dc['new'] == 1].groupby(['city']).size()
city_temp = series2df(city_new, 'city', '新设企业')
city_all = city_temp
city_all

,新设企业
city,
仙桃市,1792
十堰市,4005
咸宁市,5124
天门市,1718
孝感市,3444
宜昌市,8730
恩施州,3991
武汉市,42650
潜江市,1224


## 存量企业
存量企业包括了在这半年中注吊销和迁出的企业

In [19]:
city_live = dc.groupby(['city']).size()
city_all = df_combine(city_all, city_live, 'city', '存量企业(包含此期间内的注吊销&迁出)')
city_all

,新设企业,存量企业(包含此期间内的注吊销&迁出)
city,,
仙桃市,1792,32820
十堰市,4005,73912
咸宁市,5124,76302
天门市,1718,25129
孝感市,3444,74799
宜昌市,8730,117470
恩施州,3991,79610
武汉市,42650,766388
潜江市,1224,18065


In [20]:
city_live = dc[dc['jyzt'] == 1].groupby(['city']).size()
city_all = df_combine(city_all, city_live, 'city', '存量企业(不包含此期间内的注吊销&迁出)')
city_all

,新设企业,存量企业(包含此期间内的注吊销&迁出),存量企业(不包含此期间内的注吊销&迁出)
city,,,
仙桃市,1792,32820,32162
十堰市,4005,73912,72549
咸宁市,5124,76302,72372
天门市,1718,25129,24610
孝感市,3444,74799,72825
宜昌市,8730,117470,114174
恩施州,3991,79610,77632
武汉市,42650,766388,746052
潜江市,1224,18065,17743


## 注吊销企业

In [21]:
city_zdx = dc[(dc['tslx'] == 7) | (dc['tslx'] == 11)].groupby(['city']).size()
city_all = df_combine(city_all, city_zdx, 'city', '注吊销企业')
city_all

,新设企业,存量企业(包含此期间内的注吊销&迁出),存量企业(不包含此期间内的注吊销&迁出),注吊销企业
city,,,,
仙桃市,1792,32820,32162,658
十堰市,4005,73912,72549,1362
咸宁市,5124,76302,72372,3930
天门市,1718,25129,24610,519
孝感市,3444,74799,72825,1974
宜昌市,8730,117470,114174,3290
恩施州,3991,79610,77632,1976
武汉市,42650,766388,746052,20249
潜江市,1224,18065,17743,322


### 注销

In [22]:
city_zx = dc[dc['tslx'] == 7].groupby(['city']).size()
city_all = df_combine(city_all, city_zx, 'city', '注销企业')
city_all

,新设企业,存量企业(包含此期间内的注吊销&迁出),存量企业(不包含此期间内的注吊销&迁出),注吊销企业,注销企业
city,,,,,
仙桃市,1792,32820,32162,658,658
十堰市,4005,73912,72549,1362,1362
咸宁市,5124,76302,72372,3930,3782
天门市,1718,25129,24610,519,519
孝感市,3444,74799,72825,1974,1974
宜昌市,8730,117470,114174,3290,2653
恩施州,3991,79610,77632,1976,1810
武汉市,42650,766388,746052,20249,16641
潜江市,1224,18065,17743,322,322


### 吊销

In [23]:
city_dx = dc[dc['tslx'] == 11].groupby(['city']).size()
city_all = df_combine(city_all, city_dx, 'city', '吊销企业')
city_all

,新设企业,存量企业(包含此期间内的注吊销&迁出),存量企业(不包含此期间内的注吊销&迁出),注吊销企业,注销企业,吊销企业
city,,,,,,
仙桃市,1792,32820,32162,658,658,NaN
十堰市,4005,73912,72549,1362,1362,NaN
咸宁市,5124,76302,72372,3930,3782,148.0
天门市,1718,25129,24610,519,519,NaN
孝感市,3444,74799,72825,1974,1974,NaN
宜昌市,8730,117470,114174,3290,2653,637.0
恩施州,3991,79610,77632,1976,1810,166.0
武汉市,42650,766388,746052,20249,16641,3608.0
潜江市,1224,18065,17743,322,322,NaN


## 迁入迁出企业

In [24]:
city_qrc = dc[(dc['tslx'] == 9) | (dc['tslx'] == 13)].groupby(['city']).size()
city_all = df_combine(city_all, city_qrc, 'city', '迁入迁出企业')
city_all

,新设企业,存量企业(包含此期间内的注吊销&迁出),存量企业(不包含此期间内的注吊销&迁出),注吊销企业,注销企业,吊销企业,迁入迁出企业
city,,,,,,,
仙桃市,1792,32820,32162,658,658,NaN,64
十堰市,4005,73912,72549,1362,1362,NaN,973
咸宁市,5124,76302,72372,3930,3782,148.0,118
天门市,1718,25129,24610,519,519,NaN,49
孝感市,3444,74799,72825,1974,1974,NaN,208
宜昌市,8730,117470,114174,3290,2653,637.0,166
恩施州,3991,79610,77632,1976,1810,166.0,40
武汉市,42650,766388,746052,20249,16641,3608.0,3528
潜江市,1224,18065,17743,322,322,NaN,53


### 迁入

In [25]:
city_qr = dc[dc['tslx'] == 9].groupby(['city']).size()
city_all = df_combine(city_all, city_qr, 'hydm', '迁入企业')
city_all

,新设企业,存量企业(包含此期间内的注吊销&迁出),存量企业(不包含此期间内的注吊销&迁出),注吊销企业,注销企业,吊销企业,迁入迁出企业,迁入企业
仙桃市,1792,32820,32162,658,658,NaN,64,64
十堰市,4005,73912,72549,1362,1362,NaN,973,972
咸宁市,5124,76302,72372,3930,3782,148.0,118,118
天门市,1718,25129,24610,519,519,NaN,49,49
孝感市,3444,74799,72825,1974,1974,NaN,208,208
宜昌市,8730,117470,114174,3290,2653,637.0,166,160
恩施州,3991,79610,77632,1976,1810,166.0,40,38
武汉市,42650,766388,746052,20249,16641,3608.0,3528,3441
潜江市,1224,18065,17743,322,322,NaN,53,53
神农架林区,82,2932,2891,41,41,NaN,2,2


### 迁出

In [26]:
city_qc = dc[dc['tslx'] == 13].groupby(['city']).size()
city_all = df_combine(city_all, city_qc, 'hydm', '迁出企业')
city_all

,新设企业,存量企业(包含此期间内的注吊销&迁出),存量企业(不包含此期间内的注吊销&迁出),注吊销企业,注销企业,吊销企业,迁入迁出企业,迁入企业,迁出企业
仙桃市,1792,32820,32162,658,658,NaN,64,64,NaN
十堰市,4005,73912,72549,1362,1362,NaN,973,972,1.0
咸宁市,5124,76302,72372,3930,3782,148.0,118,118,NaN
天门市,1718,25129,24610,519,519,NaN,49,49,NaN
孝感市,3444,74799,72825,1974,1974,NaN,208,208,NaN
宜昌市,8730,117470,114174,3290,2653,637.0,166,160,6.0
恩施州,3991,79610,77632,1976,1810,166.0,40,38,2.0
武汉市,42650,766388,746052,20249,16641,3608.0,3528,3441,87.0
潜江市,1224,18065,17743,322,322,NaN,53,53,NaN
神农架林区,82,2932,2891,41,41,NaN,2,2,NaN


## 保存

In [27]:
city_all.fillna(value = 0).to_excel('./' + time_str + '/不同地区下细分.xlsx')

# 不同产业下

## 新设企业

In [28]:
three_new = dc[dc['new'] == 1].groupby(['three']).size()
three_temp = series2df(three_new, 'three', '新设企业')
three_all = three_temp
three_all

,新设企业
three,
1,6998
2,19492
3,76190


## 存量企业
存量企业包括了在这半年中注吊销和迁出的企业

In [29]:
three_live = dc.groupby(['three']).size()
three_all = df_combine(three_all, three_live, 'three', '存量企业(包含此期间内的注吊销&迁出)')
three_all

,新设企业,存量企业(包含此期间内的注吊销&迁出)
three,,
1,6998,83587
2,19492,406451
3,76190,1336985


In [30]:
three_live = dc[dc['jyzt'] == 1].groupby(['three']).size()
three_all = df_combine(three_all, three_live, 'three', '存量企业(不包含此期间内的注吊销&迁出)')
three_all

,新设企业,存量企业(包含此期间内的注吊销&迁出),存量企业(不包含此期间内的注吊销&迁出)
three,,,
1,6998,83587,82125
2,19492,406451,400223
3,76190,1336985,1298375


## 注吊销企业

In [31]:
three_zdx = dc[(dc['tslx'] == 7) | (dc['tslx'] == 11)].groupby(['three']).size()
three_all = df_combine(three_all, three_zdx, 'three', '注吊销企业')
three_all

,新设企业,存量企业(包含此期间内的注吊销&迁出),存量企业(不包含此期间内的注吊销&迁出),注吊销企业
three,,,,
1,6998,83587,82125,1461
2,19492,406451,400223,6200
3,76190,1336985,1298375,38511


### 注销

In [32]:
three_zx = dc[dc['tslx'] == 7].groupby(['three']).size()
three_all = df_combine(three_all, three_zx, 'three', '注销企业')
three_all

,新设企业,存量企业(包含此期间内的注吊销&迁出),存量企业(不包含此期间内的注吊销&迁出),注吊销企业,注销企业
three,,,,,
1,6998,83587,82125,1461,1396
2,19492,406451,400223,6200,5641
3,76190,1336985,1298375,38511,34075


### 吊销

In [33]:
three_dx = dc[dc['tslx'] == 11].groupby(['three']).size()
three_all = df_combine(three_all, three_dx, 'three', '吊销企业')
three_all

,新设企业,存量企业(包含此期间内的注吊销&迁出),存量企业(不包含此期间内的注吊销&迁出),注吊销企业,注销企业,吊销企业
three,,,,,,
1,6998,83587,82125,1461,1396,65
2,19492,406451,400223,6200,5641,559
3,76190,1336985,1298375,38511,34075,4436


## 迁入迁出企业

In [34]:
three_qrc = dc[(dc['tslx'] == 9) | (dc['tslx'] == 13)].groupby(['three']).size()
three_all = df_combine(three_all, three_qrc, 'three', '迁入迁出企业')
three_all

,新设企业,存量企业(包含此期间内的注吊销&迁出),存量企业(不包含此期间内的注吊销&迁出),注吊销企业,注销企业,吊销企业,迁入迁出企业
three,,,,,,,
1,6998,83587,82125,1461,1396,65,198
2,19492,406451,400223,6200,5641,559,1941
3,76190,1336985,1298375,38511,34075,4436,7249


### 迁入

In [35]:
three_qr = dc[dc['tslx'] == 9].groupby(['three']).size()
three_all = df_combine(three_all, three_qr, 'hydm', '迁入企业')
three_all

,新设企业,存量企业(包含此期间内的注吊销&迁出),存量企业(不包含此期间内的注吊销&迁出),注吊销企业,注销企业,吊销企业,迁入迁出企业,迁入企业
1,6998,83587,82125,1461,1396,65,198,197
2,19492,406451,400223,6200,5641,559,1941,1913
3,76190,1336985,1298375,38511,34075,4436,7249,7150


### 迁出

In [36]:
three_qc = dc[dc['tslx'] == 13].groupby(['three']).size()
three_all = df_combine(three_all, three_qc, 'hydm', '迁出企业')
three_all

,新设企业,存量企业(包含此期间内的注吊销&迁出),存量企业(不包含此期间内的注吊销&迁出),注吊销企业,注销企业,吊销企业,迁入迁出企业,迁入企业,迁出企业
1,6998,83587,82125,1461,1396,65,198,197,1
2,19492,406451,400223,6200,5641,559,1941,1913,28
3,76190,1336985,1298375,38511,34075,4436,7249,7150,99


## 保存 

In [37]:
three_all.fillna(value = 0).to_excel('./' + time_str + '/不同产业下细分.xlsx')

# 注册资金规模

## 不同行业注册资金规模

In [38]:
temp_sum = pd.DataFrame(dc.groupby(['hydm', 'scale'])['zczb'].sum())
df_industry_scale_sum = temp_sum.sum(level=0,axis=0)
industry_scale = dc.groupby(['hydm', 'scale']).size()
df_industry_scale = pd.DataFrame(industry_scale)
df_industry_scale.rename(columns={0:'count'}, inplace=True)
df_industry_scale = df_industry_scale.join(temp_sum)
df_industry_scale

count          zczb
hydm scale                     
A    1      51916  2.423418e+06
     2      26843  1.199454e+07
     3       4573  1.438649e+07
     4        165  2.435288e+06
     5         90  1.511037e+07
...           ...           ...
Z    1       7661  3.324375e+05
     2       3311  1.456277e+06
     3        612  2.145105e+06
     4         25  3.698712e+05
     5         38  2.938794e+07

[104 rows x 2 columns]

In [39]:
df_industry_scale_sum

,zczb
hydm,
A,4.635010e+07
B,8.789323e+06
C,1.747315e+08
D,4.890202e+07
E,2.195218e+08
F,1.590879e+08
G,7.642013e+07
H,7.912007e+06
I,3.325218e+07


### 保存

In [40]:
write = pd.ExcelWriter('./' + time_str + '/不同行业注册资金规模.xlsx')
df_industry_scale.fillna(value = 0).to_excel(write, sheet_name='细分')
df_industry_scale_sum.to_excel(write, sheet_name='求和')
write.close()

## 不同产业注册资金规模

In [41]:
temp_sum = pd.DataFrame(dc.groupby(['three', 'scale'])['zczb'].sum())
df_three_scale_sum = temp_sum.sum(level=0,axis=0)
three_scale = dc.groupby(['three', 'scale']).size()
df_three_scale = pd.DataFrame(three_scale)
df_three_scale.rename(columns={0:'count'}, inplace=True)
df_three_scale = df_three_scale.join(temp_sum)
df_three_scale

count          zczb
three scale                      
1     1       51916  2.423418e+06
      2       26843  1.199454e+07
      3        4573  1.438649e+07
      4         165  2.435288e+06
      5          90  1.511037e+07
2     1      176815  1.011874e+07
      2      186960  1.029623e+08
      3       39161  1.311124e+08
      4        2021  2.874585e+07
      5        1494  1.790053e+08
3     1      879610  4.154587e+07
      2      390832  1.767334e+08
      3       60030  2.090208e+08
      4        3066  4.736366e+07
      5        3447  6.413002e+08

In [42]:
df_three_scale_sum

,zczb
three,
1,4.635010e+07
2,4.519447e+08
3,1.115964e+09


### 保存

In [43]:
write = pd.ExcelWriter('./' + time_str + '/不同产业注册资金规模.xlsx')
df_three_scale.fillna(value = 0).to_excel(write, sheet_name='细分')
df_three_scale_sum.to_excel(write, sheet_name='求和')
write.close()

## 不同地区注册资金规模

In [44]:
temp_sum = pd.DataFrame(dc.groupby(['city', 'scale'])['zczb'].sum())
df_city_scale_sum = temp_sum.sum(level=0,axis=0)
city_scale = dc.groupby(['city', 'scale']).size()
df_city_scale = pd.DataFrame(city_scale)
df_city_scale.rename(columns={0:'count'}, inplace=True)
df_city_scale = df_city_scale.join(temp_sum)
df_city_scale

count          zczb
city scale                     
仙桃市  1      23539  1.146413e+06
     2       8089  3.469592e+06
     3       1082  3.698129e+06
     4         60  8.414045e+05
     5         50  5.576238e+06
...           ...           ...
黄石市  1      50157  2.169758e+06
     2      21476  1.034771e+07
     3       4039  1.366149e+07
     4        199  3.063848e+06
     5        176  1.212669e+07

[85 rows x 2 columns]

In [45]:
df_city_scale_sum

,zczb
city,
仙桃市,1.473178e+07
十堰市,6.795422e+07
咸宁市,4.108903e+07
天门市,9.743977e+06
孝感市,4.627946e+07
宜昌市,6.239245e+07
恩施州,3.453187e+07
武汉市,8.858593e+08
潜江市,1.234604e+07


### 保存

In [46]:
write = pd.ExcelWriter('./' + time_str + '/不同地区注册资金规模.xlsx')
df_city_scale.fillna(value = 0).to_excel(write, sheet_name='细分')
df_city_scale_sum.to_excel(write, sheet_name='求和')
write.close()

# 各地区不同行业下市场主体数

## 新设企业

In [47]:
c_i_new = dc[dc['new'] == 1].groupby(['city', 'hydm']).size()
c_i_temp = pd.DataFrame(c_i_new)
c_i_temp.rename(columns={0:'新设企业'}, inplace=True)
c_i_all = c_i_temp
c_i_all

新设企业
city hydm      
仙桃市  A      170
     C      364
     D        9
     E      212
     F      507
...         ...
黄石市  N       26
     O      264
     P       13
     Q       22
     R       74

[312 rows x 1 columns]

## 存量企业
存量企业包括了在这半年中注吊销和迁出的企业

In [48]:
c_i_live = dc.groupby(['city', 'hydm']).size()
c_i_temp = pd.DataFrame(c_i_live)
c_i_temp.rename(columns={0:'存量企业(包含此期间内的注吊销&迁出)'}, inplace=True)
c_i_all = pd.concat([c_i_all, c_i_temp], axis=1)
c_i_all

新设企业  存量企业(包含此期间内的注吊销&迁出)
city hydm                            
仙桃市  A     170.0                 3043
     C     364.0                 6297
     D       9.0                  139
     E     212.0                 3896
     F     507.0                10041
...          ...                  ...
鄂州市  S       NaN                   11
     Z       NaN                   29
随州市  S       NaN                   14
黄石市  S       NaN                   10
     Z       NaN                  443

[341 rows x 2 columns]

In [49]:
c_i_live = dc[dc['jyzt'] == 0].groupby(['city', 'hydm']).size()
c_i_temp = pd.DataFrame(c_i_live)
c_i_temp.rename(columns={0:'存量企业(不包含此期间内的注吊销&迁出)'}, inplace=True)
c_i_all = pd.concat([c_i_all, c_i_temp], axis=1)
c_i_all

新设企业  存量企业(包含此期间内的注吊销&迁出)  存量企业(不包含此期间内的注吊销&迁出)
city hydm                                                  
仙桃市  A     170.0                 3043                  54.0
     C     364.0                 6297                 101.0
     D       9.0                  139                   4.0
     E     212.0                 3896                  55.0
     F     507.0                10041                 226.0
...          ...                  ...                   ...
鄂州市  S       NaN                   11                   1.0
     Z       NaN                   29                   NaN
随州市  S       NaN                   14                   NaN
黄石市  S       NaN                   10                   NaN
     Z       NaN                  443                   5.0

[341 rows x 3 columns]

## 注吊销企业

In [50]:
c_i_zdx = dc[(dc['tslx'] == 7) | (dc['tslx'] == 11)].groupby(['city', 'hydm']).size()
c_i_temp = pd.DataFrame(c_i_zdx)
c_i_temp.rename(columns={0:'注吊销企业'}, inplace=True)
c_i_all = pd.concat([c_i_all, c_i_temp], axis=1)
c_i_all

新设企业  存量企业(包含此期间内的注吊销&迁出)  存量企业(不包含此期间内的注吊销&迁出)  注吊销企业
city hydm                                                         
仙桃市  A     170.0                 3043                  54.0   54.0
     C     364.0                 6297                 101.0  101.0
     D       9.0                  139                   4.0    4.0
     E     212.0                 3896                  55.0   55.0
     F     507.0                10041                 226.0  226.0
...          ...                  ...                   ...    ...
鄂州市  S       NaN                   11                   1.0    1.0
     Z       NaN                   29                   NaN    NaN
随州市  S       NaN                   14                   NaN    NaN
黄石市  S       NaN                   10                   NaN    NaN
     Z       NaN                  443                   5.0    5.0

[341 rows x 4 columns]

### 注销企业

In [51]:
c_i_zx = dc[dc['tslx'] == 7].groupby(['city', 'hydm']).size()
c_i_temp = pd.DataFrame(c_i_zx)
c_i_temp.rename(columns={0:'注销企业'}, inplace=True)
c_i_all = pd.concat([c_i_all, c_i_temp], axis=1)
c_i_all

新设企业  存量企业(包含此期间内的注吊销&迁出)  存量企业(不包含此期间内的注吊销&迁出)  注吊销企业   注销企业
city hydm                                                                
仙桃市  A     170.0                 3043                  54.0   54.0   54.0
     C     364.0                 6297                 101.0  101.0  101.0
     D       9.0                  139                   4.0    4.0    4.0
     E     212.0                 3896                  55.0   55.0   55.0
     F     507.0                10041                 226.0  226.0  226.0
...          ...                  ...                   ...    ...    ...
鄂州市  S       NaN                   11                   1.0    1.0    NaN
     Z       NaN                   29                   NaN    NaN    NaN
随州市  S       NaN                   14                   NaN    NaN    NaN
黄石市  S       NaN                   10                   NaN    NaN    NaN
     Z       NaN                  443                   5.0    5.0    5.0

[341 rows x 5 columns]

### 吊销企业

In [52]:
c_i_dx = dc[dc['tslx'] == 11].groupby(['city', 'hydm']).size()
c_i_temp = pd.DataFrame(c_i_dx)
c_i_temp.rename(columns={0:'吊销企业'}, inplace=True)
c_i_all = pd.concat([c_i_all, c_i_temp], axis=1)
c_i_all

新设企业  存量企业(包含此期间内的注吊销&迁出)  存量企业(不包含此期间内的注吊销&迁出)  注吊销企业   注销企业  \
city hydm                                                                   
仙桃市  A     170.0                 3043                  54.0   54.0   54.0   
     C     364.0                 6297                 101.0  101.0  101.0   
     D       9.0                  139                   4.0    4.0    4.0   
     E     212.0                 3896                  55.0   55.0   55.0   
     F     507.0                10041                 226.0  226.0  226.0   
...          ...                  ...                   ...    ...    ...   
鄂州市  S       NaN                   11                   1.0    1.0    NaN   
     Z       NaN                   29                   NaN    NaN    NaN   
随州市  S       NaN                   14                   NaN    NaN    NaN   
黄石市  S       NaN                   10                   NaN    NaN    NaN   
     Z       NaN                  443                   5.0    5.0    5.0   

           吊销企业  
city hydm        
仙桃市  A      NaN  
     C      NaN  
     D      NaN  
     E      NaN  
     F      NaN  
...         ...  
鄂州市  S      1.0  
     Z      NaN  
随州市  S      NaN  
黄石市  S      NaN  
     Z      NaN  

[341 rows x 6 columns]

## 迁入迁出企业

In [53]:
c_i_qrc = dc[(dc['tslx'] == 9) | (dc['tslx'] == 13)].groupby(['city', 'hydm']).size()
c_i_temp = pd.DataFrame(c_i_qrc)
c_i_temp.rename(columns={0:'迁入迁出企业'}, inplace=True)
c_i_all = pd.concat([c_i_all, c_i_temp], axis=1)
c_i_all

新设企业  存量企业(包含此期间内的注吊销&迁出)  存量企业(不包含此期间内的注吊销&迁出)  注吊销企业   注销企业  \
city hydm                                                                   
仙桃市  A     170.0                 3043                  54.0   54.0   54.0   
     C     364.0                 6297                 101.0  101.0  101.0   
     D       9.0                  139                   4.0    4.0    4.0   
     E     212.0                 3896                  55.0   55.0   55.0   
     F     507.0                10041                 226.0  226.0  226.0   
...          ...                  ...                   ...    ...    ...   
鄂州市  S       NaN                   11                   1.0    1.0    NaN   
     Z       NaN                   29                   NaN    NaN    NaN   
随州市  S       NaN                   14                   NaN    NaN    NaN   
黄石市  S       NaN                   10                   NaN    NaN    NaN   
     Z       NaN                  443                   5.0    5.0    5.0   

           吊销企业  迁入迁出企业  
city hydm                
仙桃市  A      NaN     2.0  
     C      NaN     5.0  
     D      NaN     NaN  
     E      NaN     7.0  
     F      NaN    25.0  
...         ...     ...  
鄂州市  S      1.0     NaN  
     Z      NaN     3.0  
随州市  S      NaN     NaN  
黄石市  S      NaN     NaN  
     Z      NaN     NaN  

[341 rows x 7 columns]

### 迁入企业

In [54]:
c_i_qr = dc[dc['tslx'] == 9].groupby(['city', 'hydm']).size()
c_i_temp = pd.DataFrame(c_i_qr)
c_i_temp.rename(columns={0:'迁入企业'}, inplace=True)
c_i_all = pd.concat([c_i_all, c_i_temp], axis=1)
c_i_all

新设企业  存量企业(包含此期间内的注吊销&迁出)  存量企业(不包含此期间内的注吊销&迁出)  注吊销企业   注销企业  \
city hydm                                                                   
仙桃市  A     170.0                 3043                  54.0   54.0   54.0   
     C     364.0                 6297                 101.0  101.0  101.0   
     D       9.0                  139                   4.0    4.0    4.0   
     E     212.0                 3896                  55.0   55.0   55.0   
     F     507.0                10041                 226.0  226.0  226.0   
...          ...                  ...                   ...    ...    ...   
鄂州市  S       NaN                   11                   1.0    1.0    NaN   
     Z       NaN                   29                   NaN    NaN    NaN   
随州市  S       NaN                   14                   NaN    NaN    NaN   
黄石市  S       NaN                   10                   NaN    NaN    NaN   
     Z       NaN                  443                   5.0    5.0    5.0   

           吊销企业  迁入迁出企业  迁入企业  
city hydm                      
仙桃市  A      NaN     2.0   2.0  
     C      NaN     5.0   5.0  
     D      NaN     NaN   NaN  
     E      NaN     7.0   7.0  
     F      NaN    25.0  25.0  
...         ...     ...   ...  
鄂州市  S      1.0     NaN   NaN  
     Z      NaN     3.0   3.0  
随州市  S      NaN     NaN   NaN  
黄石市  S      NaN     NaN   NaN  
     Z      NaN     NaN   NaN  

[341 rows x 8 columns]

### 迁出企业

In [55]:
c_i_qc = dc[dc['tslx'] == 13].groupby(['city', 'hydm']).size()
c_i_temp = pd.DataFrame(c_i_qc)
c_i_temp.rename(columns={0:'迁出企业'}, inplace=True)
c_i_all = pd.concat([c_i_all, c_i_temp], axis=1)
c_i_all

新设企业  存量企业(包含此期间内的注吊销&迁出)  存量企业(不包含此期间内的注吊销&迁出)  注吊销企业   注销企业  \
city hydm                                                                   
仙桃市  A     170.0                 3043                  54.0   54.0   54.0   
     C     364.0                 6297                 101.0  101.0  101.0   
     D       9.0                  139                   4.0    4.0    4.0   
     E     212.0                 3896                  55.0   55.0   55.0   
     F     507.0                10041                 226.0  226.0  226.0   
...          ...                  ...                   ...    ...    ...   
鄂州市  S       NaN                   11                   1.0    1.0    NaN   
     Z       NaN                   29                   NaN    NaN    NaN   
随州市  S       NaN                   14                   NaN    NaN    NaN   
黄石市  S       NaN                   10                   NaN    NaN    NaN   
     Z       NaN                  443                   5.0    5.0    5.0   

           吊销企业  迁入迁出企业  迁入企业  迁出企业  
city hydm                            
仙桃市  A      NaN     2.0   2.0   NaN  
     C      NaN     5.0   5.0   NaN  
     D      NaN     NaN   NaN   NaN  
     E      NaN     7.0   7.0   NaN  
     F      NaN    25.0  25.0   NaN  
...         ...     ...   ...   ...  
鄂州市  S      1.0     NaN   NaN   NaN  
     Z      NaN     3.0   3.0   NaN  
随州市  S      NaN     NaN   NaN   NaN  
黄石市  S      NaN     NaN   NaN   NaN  
     Z      NaN     NaN   NaN   NaN  

[341 rows x 9 columns]

## 保存

In [56]:
c_i_all.fillna(value = 0).to_excel('./' + time_str + '/各地区不同行业下市场主体数量.xlsx')